# Heat Parameter Calibration
Refer to the publication for more details on the implementation

----------------------------------------------------------------------------------

In [1]:
# import required libraries:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import eig

-----------------------------------------------------------------------------------------
# 1st algorithmic stage
-----------------------------------------------------------------------------------------

# 1) Import the temperature evidence

For any set of points in the spatial domain (x,y,z) report the corresponding temperature evidence, that is, the temperature as recorded from the experiment. If points on the fusion boundary are included, set their temperature equal to the material's melt point.

In [2]:
# convert your csv file into a dataframe, write the correct path name:

t_evidence = pd.read_csv('YOUR PATH') 

# 2) Import the simulated temperature fields

Start from an initial estimation of heat parameter vector and run a thermal simulation.

Then change each heat parameter (e.g. heat source radii, heat input efficiency, heat loss coefficients) value with a step up or down, and record all simulated temperature results in .csv files.

In [ ]:
# convert your .csv files into dataframes, write the correct path name for each file:

t_estimated = pd.read_csv('YOUR PATH') 
ra_down = pd.read_csv('YOUR PATH')
ra_up = pd.read_csv('YOUR PATH')
rl_down = pd.read_csv('YOUR PATH')
rl_up = pd.read_csv('YOUR PATH')
rv_down = pd.read_csv('YOUR PATH')
rv_up = pd.read_csv('YOUR PATH')
ef_down = pd.read_csv('YOUR PATH')
ef_up = pd.read_csv('YOUR PATH')
e_down = pd.read_csv('YOUR PATH')
e_up = pd.read_csv('YOUR PATH')
h_down = pd.read_csv('YOUR PATH')
h_up = pd.read_csv('YOUR PATH')

# 3) Calculated the partial derivatives of the simulated temperature fields

In [ ]:
# change step-size according to what you used to run the simulations:

step = 1 

# calculate the partial derivatives:

d_ra = ra_up.copy()
d_ra['temperature'] = (ra_up.copy()['temperature'] - ra_down.copy()['temperature'])/step
d_ra.rename(columns = {'temperature':'derivative'}, inplace = True)

d_rl = rl_up.copy()
d_rl['temperature'] = (rl_up.copy()['temperature'] - rl_down.copy()['temperature'])/step
d_rl.rename(columns = {'temperature':'derivative'}, inplace = True)

d_rv = rv_up.copy()
d_rv['temperature'] = (rv_up.copy()['temperature'] - rv_down.copy()['temperature'])/step
d_rv.rename(columns = {'temperature':'derivative'}, inplace = True)

d_ef = ef_up.copy()
d_ef['temperature'] = (ef_up.copy()['temperature'] - ef_down.copy()['temperature'])/step
d_ef.rename(columns = {'temperature':'derivative'}, inplace = True)

d_e = e_up.copy()
d_e['temperature'] = (e_up.copy()['temperature'] - e_down.copy()['temperature'])/step
d_e.rename(columns = {'temperature':'derivative'}, inplace = True)

d_h = h_up.copy()
d_h['temperature'] = (h_up.copy()['temperature'] - h_down.copy()['temperature'])/step
d_h.rename(columns = {'temperature':'derivative'}, inplace = True)


# 4) Calculate the error term e_bar

In [ ]:
# set appropriate values of alpha  (maximum deviations from initially estimated parameter values):

a_ra = 3
a_rl = 3
a_rv = 3
a_ef = 20
a_e = 3
a_h =  8

# calculate values of e_bar:

e_bar_ra = d_ra.copy()
e_bar_ra['derivative'] = (abs(d_h['derivative'])*(a_h) + abs(d_ef['derivative'])*(a_ef) + abs(d_e['derivative'])*(a_e) + abs(d_rl['derivative'])*(a_rl) + abs(d_rv['derivative'])*(a_rv))/abs(d_ra['derivative'])
e_bar_ra.rename(columns = {'derivative':'e_bar'}, inplace = True)

e_bar_rl = d_rl.copy()
e_bar_rl['derivative'] = (abs(d_h['derivative'])*(a_h) + abs(d_ef['derivative'])*(a_ef) + abs(d_e['derivative'])*(a_e) + abs(d_ra['derivative'])*(a_ra) + abs(d_rv['derivative'])*(a_rv))/abs(d_rl['derivative'])
e_bar_rl.rename(columns = {'derivative':'e_bar'}, inplace = True)

e_bar_rv = d_rv.copy()
e_bar_rv['derivative'] = (abs(d_h['derivative'])*(a_h) + abs(d_ef['derivative'])*(a_ef) + abs(d_e['derivative'])*(a_e) + abs(d_rl['derivative'])*(a_rl) + abs(d_ra['derivative'])*(a_ra))/abs(d_rv['derivative'])
e_bar_rv.rename(columns = {'derivative':'e_bar'}, inplace = True)

e_bar_ef = d_ef.copy()
e_bar_ef['derivative'] = (abs(d_h['derivative'])*(a_h) + abs(d_e['derivative'])*(a_e) + abs(d_ra['derivative'])*(a_ra) + abs(d_rl['derivative'])*(a_rl) + abs(d_rv['derivative'])*(a_rv))/abs(d_ef['derivative'])
e_bar_ef.rename(columns = {'derivative':'e_bar'}, inplace = True)

e_bar_e = d_e.copy()
e_bar_e['derivative'] = (abs(d_h['derivative'])*(a_h) + abs(d_ef['derivative'])*(a_ef) + abs(d_ra['derivative'])*(a_ra) + abs(d_rl['derivative'])*(a_rl) + abs(d_rv['derivative'])*(a_rv))/abs(d_e['derivative'])
e_bar_e.rename(columns = {'derivative':'e_bar'}, inplace = True)

e_bar_h = d_h.copy()
e_bar_h['derivative'] = (abs(d_e['derivative'])*(a_e) + abs(d_ef['derivative'])*(a_ef) + abs(d_ra['derivative'])*(a_ra) + abs(d_rl['derivative'])*(a_rl) + abs(d_rv['derivative'])*(a_rv))/abs(d_h['derivative'])
e_bar_h.rename(columns = {'derivative':'e_bar'}, inplace = True)

# 5) Estimate the new heat parameter values

In [ ]:
# parameter value of initialisation / previous iteration, change as necessary:
ra_prev = 4 

der_ra = d_ra.copy()
p_ra = t_evidence.copy()
p_ra['temperature'] = ra_prev + (t_evidence['temperature'] - t_estimated['temperature'])/der_ra['derivative']  
p_ra.rename(columns = {'temperature':'ra'}, inplace = True)
p_ra['e_bar'] = e_bar_ra['e_bar'].copy()
p_ra['derivative'] = der_ra['derivative'].copy()

# CHANGE THE THRESHOLD IF NECESSARY TO INCLUDE THE DESIRED NUMBER OF POINTS FOR THE ESTIMATION
threshold = 0.5 
filtered_p_ra = p_ra[(p_ra['e_bar'] <= threshold)].copy() 
filtered_p_ra.reset_index(drop=True, inplace=True)
ra = round(filtered_p_ra['ra'].median(),0)
print("median is:",ra)
filtered_p_ra['ra'].plot(kind='hist', bins=80, range=(0, 10))

In [ ]:
# parameter value of initialisation / previous iteration , change as necessary:
rv_prev = 4

der_rv = d_rv.copy()
p_rv = t_evidence.copy()
p_rv['temperature'] = rv_prev + (t_evidence['temperature'] - t_estimated['temperature'])/der_rv['derivative']  
p_rv.rename(columns = {'temperature':'rv'}, inplace = True)
p_rv['e_bar'] = e_bar_rv['e_bar'].copy()
p_rv['derivative'] = der_rv['derivative'].copy()

# CHANGE THE THRESHOLD IF NECESSARY TO INCLUDE THE DESIRED NUMBER OF POINTS FOR THE ESTIMATION
threshold = 0.5 
filtered_p_rv = p_rv[(p_rv['e_bar'] <= threshold)].copy()
filtered_p_rv.reset_index(drop=True, inplace=True)
rv = round(filtered_p_rv['rv'].median(),0)
print("median is:",rv)
filtered_p_rv['rv'].plot(kind='hist', bins=80, range=(0, 10))

In [ ]:
# parameter value of initialisation / previous iteration , change as necessary:
rl_prev = 4

der_rl = d_rl.copy()
p_rl = t_evidence.copy()
p_rl['temperature'] = rl_prev + (t_evidence['temperature'] - t_estimated['temperature'])/der_rl['derivative']  
p_rl.rename(columns = {'temperature':'rl'}, inplace = True)
p_rl['e_bar'] = e_bar_rl['e_bar'].copy()
p_rl['derivative'] = der_rl['derivative'].copy()

# CHANGE THE THRESHOLD IF NECESSARY TO INCLUDE THE DESIRED NUMBER OF POINTS FOR THE ESTIMATION
threshold = 0.5 
filtered_p_rl = p_rl[(p_rl['e_bar'] <= threshold)].copy()
filtered_p_rl.reset_index(drop=True, inplace=True)
rl = round(filtered_p_rl['rl'].median(),0)
print("median is:",rl)
filtered_p_rl['rl'].plot(kind='hist', bins=80, range=(0, 10))

In [ ]:
# parameter value of initialisation / previous iteration , change as necessary:
ef_prev = 80

der_ef = d_ef.copy()
p_ef = t_evidence.copy()
p_ef['temperature'] = ef_prev + (t_evidence['temperature'] - t_estimated['temperature']) / der_ef['derivative']
p_ef.rename(columns = {'temperature':'efficiency'}, inplace = True)
p_ef['e_bar'] = e_bar_ef['e_bar'].copy()
p_ef['derivative'] = der_ef['derivative'].copy()

# CHANGE THE THRESHOLD IF NECESSARY TO INCLUDE THE DESIRED NUMBER OF POINTS FOR THE ESTIMATION
threshold = 0.5 
filtered_p_ef = p_ef[(p_ef['e_bar'] <= threshold)].copy()
filtered_p_ef.reset_index(drop=True, inplace=True)
ef = round(filtered_p_ef['efficiency'].median(),0)
print("median is:",ef)
filtered_p_ef['efficiency'].plot(kind='hist', bins=80, range=(60, 100))

In [ ]:
# parameter value of initialisation / previous iteration , change as necessary:
e_prev = 7

der_e = d_e.copy()
p_e = t_evidence.copy()
p_e['temperature'] = e_prev + (t_evidence['temperature'] - t_estimated['temperature']) / der_e['derivative']
p_e.rename(columns = {'temperature':'e'}, inplace = True)
p_e['e_bar'] = e_bar_e['e_bar'].copy()
p_e['derivative'] = der_e['derivative'].copy()

# CHANGE THE THRESHOLD IF NECESSARY TO INCLUDE THE DESIRED NUMBER OF POINTS FOR THE ESTIMATION
threshold = 0.5 
filtered_p_e = p_e[(p_e['e_bar'] <= threshold)].copy()
filtered_p_e.reset_index(drop=True, inplace=True)
e = round(filtered_p_e['e'].median(),0)
print("median is:",e)
filtered_p_e['e'].plot(kind='hist', bins=80, range=(0, 10))

In [ ]:
# parameter value of initialisation / previous iteration , change as necessary:
h_prev = 13 

der_h = d_h.copy()
p_h = t_evidence.copy()
p_h['temperature'] = h_prev + (t_evidence['temperature'] - t_estimated['temperature']) / der_h['derivative']
p_h.rename(columns = {'temperature':'h'}, inplace = True)
p_h['e_bar'] = e_bar_h['e_bar'].copy()
p_h['derivative'] = der_h['derivative'].copy()

# CHANGE THE THRESHOLD IF NECESSARY TO INCLUDE THE DESIRED NUMBER OF POINTS FOR THE ESTIMATION
threshold = 0.5 
filtered_p_h = p_h[(p_h['e_bar'] <= threshold)].copy()
filtered_p_h.reset_index(drop=True, inplace=True)
h = round(filtered_p_h['h'].median(),0)
print("median is:",h)
filtered_p_h['h'].plot(kind='hist', bins=80, range=(0, 20))

# 6) Repeat steps 2 to 5 until convergence

If the uncertainty interval found from the 1st algorithmic stage is wide, progress to the 2nd algorithmic stage.

-----------------------------------------------------------------------------------------
# 2nd algorithmic stage
-----------------------------------------------------------------------------------------

# 7) Find a particular solution from the interval found in the 1st algorithmic stage

# 8) Find the partial derivatives on this particular solution (as in steps 2&3)

# 9) Calculate the Jacobian matrix

In [ ]:
column_names = ['d_rl', 'd_rv','d_ra','d_ef','d_e', 'd_h']

Ja = pd.concat([
    d_rl['derivative'],
    d_rv['derivative'],
    d_ra['derivative'],
    d_ef['derivative'],
    d_e['derivative'],
    d_h['derivative']
], axis=1)

Ja.columns = column_names

Ja.reset_index(drop=True, inplace=True)
Ja

# 10) Find the eigenvector/s which correspond to the minimum eigenvalue/s

In [ ]:
JJa = np.matmul(Ja.transpose(),Ja) 

eigenvalues, eigenvectors = eig(JJa) # the ith column of eigenvectors corresponds to ith eigenvalue
print(eigenvalues)
print(eigenvectors)